## Detailed Article Explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/541732/paris-olympics-ticket-information-chatbot-with-memory-using-langchain

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Installing and Importing Required Libraries

In [1]:
!pip install -U langchain
!pip install langchain-openai
!pip install pypdf
!pip install faiss-cpu

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
import os

## Paris Olympic Chatbot for Generating a Single Response

In [2]:
openai_key = os.environ.get('OPENAI_KEY2')

In [3]:
llm = ChatOpenAI(
    openai_api_key = openai_key ,
    model = 'gpt-4',
    temperature = 0.5
)

In [4]:
loader = PyPDFLoader("https://tickets.paris2024.org/obj/media/FR-Paris2024/ticket-prices.pdf")
docs = loader.load_and_split()

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key = openai_key)
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

Question: {input}

Context: {context}
"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [8]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [9]:
def generate_response(query):
    response = retrieval_chain.invoke({"input": query})
    print(response["answer"])

In [10]:
query = "What is the lowest ticket price for tennis games?"
generate_response(query)


The lowest ticket price for tennis games is €30.


In [11]:
query = "And for beach volleyball?"
generate_response(query)

Beach Volleyball is played by two teams of two players each. They face off in the best of three sets on a sand court that is 16m long and 8m wide. The net is at the same height as indoor volleyball (2.24m for women and 2.43m for men). The game is contested by playing two sets to 21 points, and teams must win at least two points more than their opponents to win the set. If needed, the third set is played to 15 points. The matches take place at the Eiffel Tower Stadium in Paris.


In [12]:
query = "And what is the category of this ticket?"
generate_response(query)

The context does not provide specific information on the category of the ticket.


## Adding Memory to Paris Olympic Chatbot 

In [13]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

history_retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [41]:
chat_history = [
    HumanMessage(content="What is the lowest ticket price for tennis games?"), 
    AIMessage(content="The lowest ticket price for tennis games is €30.")
]

result = history_retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "And for Beach Volleyball?"
})

result[0]

Document(page_content='BEACH VOLLEYBALL (CONTINUED)BEACH VOLLEYBALL (CONTINUED)\nDATE TIMETYPE OF\nSESSIONSESSION \nDESCRIPTIONPRICE PER CATEGORY\nFIRST A B C DSESSION\nCODE\nVBV33 06/08/245:00 pm\n7:00 pmQualifications M/W - 1/4 Final ( 2 matches) € 200  € 145 € 95 € 50\nVBV34 06/08/249:00 pm\n11:00 pmQualifications M/W - 1/4 Final ( 2 matches) € 240  € 175 € 125 € 70\nVBV35 07/08/245:00 pm\n7:00 pmQualifications M/W - 1/4 Final ( 2 matches) € 200  € 145 € 95 € 50\nVBV36 07/08/249:00 pm\n11:00 pmQualifications M/W - 1/4 Final ( 2 matches) € 240  € 175 € 125 € 70\nVBV37 08/08/245:00 pm\n7:00 pmQualifications M/W - Semi-finals (2 matches) € 280 € 205 € 145 € 85\nVBV38 08/08/249:00 pm\n11:00 pmQualifications M/W - Semi-finals (2 matches) € 290 € 215 € 155 € 90\nVBV39 09/08/249:00 pm\n12:00 amMedals W - Bronze, Final € 420 € 305 € 205 € 100\nVBV40 10/08/249:00 pm\n12:00 amMedals M - Bronze, Final € 420 € 305 € 205 € 100\n* Please refer to page 5 in this document for more information on ac

In [40]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])
document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
retrieval_chain = create_retrieval_chain(history_retriever_chain, document_chain)

In [42]:
result = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "And for Beach Volleyball?"
})

print(result['answer'])

The lowest ticket price for Beach Volleyball games is €24.


## Putting it All Together - A Command Line Chatbot

In [28]:
chat_history = []

def generate_response_with_memory(query):
    result = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": query
    })

    response = result['answer']
    chat_history.extend([HumanMessage(content = query),
                       AIMessage(content = response)])
    
    return response

In [37]:
print("=======================================================================")
print("Welcome to Paris Olympics Ticket Information Chatbot. Enter your query")
print("=======================================================================")

query = ""
while query != "bye":
    query = input("\033[1m User >>:\033[0m")
    
    if query == "bye":
        chat_history = []
        print("\033[1m Chatbot>>:\033[0m Thank you for your messages, have a good day!")
        break
    response = generate_response_with_memory(query)
    print(f"\033[1m Chatbot>>:\033[0m {response}")


Welcome to Paris Olympics Ticket Information Chatbot. Enter your query
 User >>:What is the lowest ticket price for tennis games?
 Chatbot>>: The lowest ticket price for tennis games is €30.
 User >>:And for beach volleyball?
 Chatbot>>: The lowest ticket price for beach volleyball games is €24.
 User >>:And what is the category of this ticket?
 Chatbot>>: The category of this ticket is D.
 User >>:bye
 Chatbot>>: Thank you for your messages, have a good day!
